In [1]:
"""
Feature: surgical_admission
Extract from procedures_icd and discharge notes
"""
import pandas as pd
import os
import re
from config_paths import *
from utils import drop_columns

In [3]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load procedures
d_icd_procedures = pd.read_csv(os.path.join(hosp_path, "d_icd_procedures.csv"))
procedures = pd.read_csv(
    os.path.join(hosp_path, "procedures_icd.csv"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"]
)
procedures["icd_code"] = procedures["icd_code"].astype(str)
procedures["icd_version"] = procedures["icd_version"].astype(int)

# Identify surgery codes
d_icd_procedures["icd_code"] = d_icd_procedures["icd_code"].astype(str)

# ICD-10-PCS Surgery: 7 chars and starts with '0'
icd10_surgery = d_icd_procedures[
    (d_icd_procedures["icd_code"].str.len() == 7) &
    (d_icd_procedures["icd_code"].str.startswith("0"))
]

# ICD-9 Surgery: numeric and <= 86
icd9_numeric = d_icd_procedures["icd_code"].str.extract(r'^(\d+)')[0]
icd9_surgery = d_icd_procedures[
    (d_icd_procedures["icd_code"].str.len() <= 5) &
    (icd9_numeric.astype(float, errors="ignore") <= 86)
]

all_surgery_codes = pd.concat([
    icd10_surgery[["icd_code"]],
    icd9_surgery[["icd_code"]]
]).drop_duplicates()
all_surgery_codes_set = set(all_surgery_codes["icd_code"])

# Flag surgeries in procedures
procedures["is_surgery"] = procedures["icd_code"].isin(all_surgery_codes_set)
surgery_by_hadm_from_proc = (
    procedures.groupby("hadm_id")["is_surgery"]
    .any()
    .reset_index()
    .rename(columns={"is_surgery": "surgical_from_procedures"})
)

# Load discharge notes
discharge = pd.read_csv(
    os.path.join(note_path, "discharge.csv"),
    usecols=["subject_id", "hadm_id", "text"]
)

surgical_regex = r"\b(?:\ post[-\s]?op|\ post[-\s]?operative|\ status[-\s]?post|\ s/p|\ underwent|\ taken\s+to\s+(?:the\s+)?or|\ operative|\ operation|\ procedure\s+performed|\ exploratory|\ lap\b|\ laparoscop|\ laparotomy|\ appendectom|\ cholecystectom|\ prostatectom|\ hysterectom|\ nephrectom|\ colectom|\ craniotom|\ thoracotom|\ bypass|\ cabg|\ stent\s+placed|\ orif|\ fixation|\ resection|\ excision|\ debridement\ )\b"

discharge["surgical_from_notes"] = discharge["text"].str.contains(
    surgical_regex, case=False, na=False, regex=True
)

surgery_by_hadm_from_notes = (
    discharge.groupby("hadm_id")["surgical_from_notes"]
    .any()
    .reset_index()
)

# Merge
df = df.merge(surgery_by_hadm_from_proc, on="hadm_id", how="left")
df = df.merge(surgery_by_hadm_from_notes, on="hadm_id", how="left")

# Safe boolean handling
df["surgical_from_procedures"] = (
    df["surgical_from_procedures"].astype("boolean").fillna(False).astype(bool)
)
df["surgical_from_notes"] = (
    df["surgical_from_notes"].astype("boolean").fillna(False).astype(bool)
)

# Final flag
df["surgical_admission"] = (
    df["surgical_from_procedures"] | df["surgical_from_notes"]
)

# Drop intermediate columns
df.drop(columns=["surgical_from_procedures", "surgical_from_notes"], inplace=True)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'surgical_admission' added. True count: {df['surgical_admission'].sum()}")
print(f"Dataset shape: {df.shape}")


Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 53)
Feature 'surgical_admission' added. True count: 125536
Dataset shape: (158020, 53)


In [6]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [7]:
# cols_to_drop = [
#  "surgical_admission"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)